In [2]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
from time import sleep
import pandas as pd
import sqlite3
import time
import re
from datetime import date, timedelta


REMOVE_ADS_SCRIPT = """
var ads = document.querySelectorAll('[id^="google_ads"], [class*="ads"], [class*="sponsored"]');
ads.forEach(ad => ad.remove());
"""


def start_google_chrome(url, screen_width=2200, screen_height=1900):
    """
    Undetected Chrome を起動し、指定URLにアクセス
    - ウィンドウを左半分に配置
    - 広告対策オプションを追加
    :param url: 開くURL
    :param screen_width: 画面の横幅 (デフォルト: 2200)
    :param screen_height: 画面の高さ (デフォルト: 1900)
    :return: WebDriver インスタンス
    """

    # Chromeオプションを設定
    options = uc.ChromeOptions()
    options.add_argument("--no-sandbox")  # セキュリティサンドボックス無効化
    options.add_argument("--disable-blink-features=AutomationControlled")  # ボット検出回避
    options.add_argument("--disable-extensions")  # 拡張機能無効化
    options.add_argument("--disable-popup-blocking")  # ポップアップブロック無効化
    options.add_argument("--disable-gpu")  # GPUレンダリング無効化
    options.add_argument("--start-maximized")  # 最大化
    options.add_argument("--disable-infobars")  # "Chromeは自動テストソフトウェアによって制御されています"を非表示
    options.add_argument("--disable-notifications")  # 通知をブロック
    options.add_argument("--remote-debugging-port=9222")  # デバッグ用ポート

    # Chromeドライバーの起動
    driver = uc.Chrome(options=options)

    # 画面の左半分にウィンドウを配置
    driver.set_window_position(0, 0)  # 左上 (0,0) に配置
    driver.set_window_size(screen_width // 2, screen_height)  # 画面の半分の幅に設定

    # 指定URLを開く
    driver.get(url)
    print("-" * 50)
    print(f"実行成功: {url}")
    driver.implicitly_wait(30)  # 最大30秒待機
    time.sleep(1)  # 読み込み待機

    return driver  # WebDriver インスタンスを返す



def click_date_link(driver, DAYS_AGO, MAX_RETRIES=3):
    """
    指定したDAYS_AGOの日付リンクをクリックし、ページ遷移を確認する
    :param driver: Selenium WebDriver
    :param DAYS_AGO: 過去の日数（1=最新の日付）
    :param MAX_RETRIES: 最大リトライ回数
    :return: 成功した場合はTrue、失敗した場合はFalse
    """
    retries = 0
    current_url = driver.current_url

    try:
        wait = WebDriverWait(driver, 10)
        date_links = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".table-data-cell a")))

        if DAYS_AGO > len(date_links):
            print(f"エラー: DAYS_AGO={DAYS_AGO} に対応するリンクがありません。")
            return False

        target_link = date_links[DAYS_AGO - 1]
        print(f"クリック対象リンク: {target_link.text}")

        while retries < MAX_RETRIES:
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'auto', block: 'center'});", target_link)
            time.sleep(2)  # スクロール後の待機
            wait.until(EC.element_to_be_clickable(target_link))
            time.sleep(2)

            target_link.click()

            wait.until(EC.url_changes(current_url))
            time.sleep(2)

            if "google_vignette" in driver.current_url:
                retries += 1
                print(f"⚠ Google広告が表示されました。リトライします ({retries}/{MAX_RETRIES})")
                driver.back()  # 広告ページから戻る
                time.sleep(2)
                continue  # リトライ

            print(f"遷移成功: {driver.current_url}")
            date, hall_name = extract_date_hall(driver.current_url)
            return date, hall_name

    except Exception as e:
        print(f"エラー発生: {e}")
    
    print("クリック失敗: リトライ回数超過")
    return False  # 失敗


def extract_date_hall(url):
    """
    URL から日付 (YYYY-MM-DD) とホール名を抽出
    :param url: 解析するURL
    :return: (date, hall_name) のタプル
    """
    pattern = r"https://ana-slo\.com/(\d{4})-(\d{1,2})-(\d{1,2})-(.+)-data/"
    match = re.search(pattern, url)

    if match:
        year, month, day, hall_name = match.groups()
        date = f"{year}-{int(month):02d}-{int(day):02d}"  # 月日をゼロ埋め
        hall_name = hall_name.replace("-", "_").upper()  # "-" をスペースに置き換えて大文字に
        print(f"日付: {date}")
        print(f"ホール名: {hall_name}")
        return date, hall_name
    else:
        return None, None  # マッチしない場合

    
def click_machine_by_name(driver, search_machine):
    """
    指定した機種名を検索し、該当する要素をクリック
    :param driver: Selenium WebDriver
    :param search_machine: 検索する機種名
    :return: クリック成功: True / 失敗: False
    """
    try:
        title = driver.title
        date_pattern = r"\d{4}/\d{2}/\d{2}"
        date_match = re.search(date_pattern, title)
        date = date_match.group().replace("/", "-") if date_match else None
        hall_pattern = r"\d{4}/\d{2}/\d{2} (.+?) データまとめ"
        hall_match = re.search(hall_pattern, title)
        hall_name = hall_match.group(1).replace(" ", "_") if hall_match else None

        print(f"日付: {date}")
        print(f"ホール名: {hall_name}")
        
        sections = driver.find_elements(By.CLASS_NAME, "unit_get_medals_table a")
        for i, section in enumerate(sections):
            if search_machine in section.text:
                machine_name = section.text
                section_num = i
                # print(f"{i}: {machine_name}")
                driver.execute_script("arguments[0].scrollIntoView({behavior: 'auto', block: 'end'});", section)
                sleep(3)
                section.click()
                print(f"クリック成功: {machine_name}")
                return section_num, date, hall_name
            
        print("指定の機種が見つかりませんでした。")
        return False
    
    except Exception as e:
        print(f"❌ エラー発生: {e}")
        return False


def extract_and_save_model_data(driver, section_num, hall_name, date):
    """
    指定した機種のデータを抽出し、CSVに保存する
    :param driver: Selenium WebDriver
    :param section_num: セクション番号 (該当機種のインデックス)
    :param hall_name: ホール名
    :param date: 日付 (yyyy-mm-dd)
    :return: 成功時 True, 失敗時 False
    """
    try:
        # 機種名を取得
        model_name = driver.find_element(By.ID, f"section{section_num}").text
        
        # データ行を取得
        rows = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, f"#tab01_{section_num} div table tbody tr"))
        )
        
        if not rows:
            print(f"⚠️ データが見つかりません: {model_name}")
            return False

        print(f"{model_name}: {len(rows)} row 取得開始")
        print("取得中...")

        # ヘッダー取得
        header_cells = rows[0].find_elements(By.TAG_NAME, "th")
        columns = [cell.text for cell in header_cells]

        # データ取得
        data = []
        for row in rows[1:-1]:  # 最終行を除外
            cells = row.find_elements(By.TAG_NAME, "td")
            data.append([cell.text for cell in cells])

        print(f"{model_name}: {len(data)} data 取得完了")

        # DataFrameを作成しCSVに保存
        df = pd.DataFrame(data, columns=columns)
        file_name = f"../csv/{hall_name}_{model_name}_{date}.csv"
        df.to_csv(file_name, index=False, encoding="utf-8-sig")
        
        print(rows[1].text)

        print(f"データ保存完了: {file_name}")
        return True

    except Exception as e:
        print(f"エラー発生: {e}")
        return False

In [ ]:
URL = f"https://ana-slo.com/"
DATA = "ホールデータ"
PREF = "宮城県"
url = f"https://ana-slo.com/{DATA}/{PREF}/"

MAX_RETRIES = 5

In [11]:
# テストコード
driver = start_google_chrome("https://google.com")

--------------------------------------------------
実行成功: https://google.com


In [14]:
driver.get(url)
driver.execute_script(REMOVE_ADS_SCRIPT)

# date, hall_name = click_date_link(driver, DAYS_AGO, MAX_RETRIES=MAX_RETRIES)
# driver.execute_script(REMOVE_ADS_SCRIPT)

# section_num, date, hall_name = click_machine_by_name(driver, SEARCH_MACHINE)
# extract_and_save_model_data(driver, 0, hall_name, date)

In [15]:
# ホール名取得
halls = {}

table_rows = driver.find_elements(By.CLASS_NAME, "table-row a")
tmp = {}
for i, row in enumerate(table_rows):
    tmp[f"{i}"] = row.text
    if row.text == "": break
    # if i == 1: break
halls[PREF] = tmp


In [16]:
halls

{'宮城県': {'0': '21SEIKI仙台愛子',
  '1': '21SEIKI仙台泉',
  '2': '21SEIKI石巻中里',
  '3': 'ARISE DAIAN 築館店',
  '4': 'ARISE DAIAN 荒巻店',
  '5': 'chaos佐沼店',
  '6': 'chaos古川店',
  '7': 'DON+',
  '8': 'Dステーション仙台東店',
  '9': 'Dステーション仙台泉店',
  '10': 'JOYPARK湊店',
  '11': 'JOYPARK石巻大街道店(10スロ)',
  '12': 'JOYPARK赤井店',
  '13': 'P-STATION&IIスロット館',
  '14': 'SS ZAP多賀城店',
  '15': 'WINS気仙沼',
  '16': 'アムズガーデン仙台駅前店',
  '17': 'アムズガーデン利府店',
  '18': 'アムズガーデン南吉成店',
  '19': 'アムズガーデン古川店',
  '20': 'アムズガーデン幸町店',
  '21': 'アムズガーデン石巻店',
  '22': 'アムズガーデン石巻湊店',
  '23': 'ウイングあすと長町店',
  '24': 'ウイング利府店',
  '25': 'ウイング石巻店',
  '26': 'オータ古川店',
  '27': 'スーパーメルヘンワールド幸町店',
  '28': 'スーパーメルヘンワールド石巻店',
  '29': 'スロットLEO',
  '30': 'スロットインパラディソ名取',
  '31': 'スロットハウスムサシ矢本店',
  '32': 'スロットメルヘンワールド長町南店',
  '33': 'セントラル神山店',
  '34': 'つばめザ・ウィング',
  '35': 'つばめザ・ムーブ',
  '36': 'ハードロック仙台一番町店',
  '37': 'パーラーJ-遊佐沼みなみ店',
  '38': 'パーラーJ-遊佐沼店',
  '39': 'パチンコ22大和店',
  '40': 'パチンコダイイチ気仙沼店',
  '41': 'パチンコタイガー多賀城店',
  '42': 'パチンコタイガー松森店',
  '43': 'パチンコタイガー苦竹店2',
